<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#BeautifulSoup" data-toc-modified-id="BeautifulSoup-1">BeautifulSoup</a></span><ul class="toc-item"><li><span><a href="#BeautifulSoup-설치" data-toc-modified-id="BeautifulSoup-설치-1.1">BeautifulSoup 설치</a></span></li><li><span><a href="#BeautifulSoup-import-하기" data-toc-modified-id="BeautifulSoup-import-하기-1.2">BeautifulSoup import 하기</a></span></li><li><span><a href="#BeautifulSoup-객체-생성-및-parser-사용" data-toc-modified-id="BeautifulSoup-객체-생성-및-parser-사용-1.3">BeautifulSoup 객체 생성 및 parser 사용</a></span></li><li><span><a href="#jupyter-notebook-Magic-Command-:-whos" data-toc-modified-id="jupyter-notebook-Magic-Command-:-whos-1.4">jupyter notebook Magic Command : whos</a></span></li><li><span><a href="#Selector-기본" data-toc-modified-id="Selector-기본-1.5">Selector 기본</a></span></li><li><span><a href="#Selector-실습" data-toc-modified-id="Selector-실습-1.6">Selector 실습</a></span><ul class="toc-item"><li><span><a href="#html.parser" data-toc-modified-id="html.parser-1.6.1">html.parser</a></span></li><li><span><a href="#lxml-parser" data-toc-modified-id="lxml-parser-1.6.2">lxml parser</a></span></li><li><span><a href="#부모자식-tag-선택하기(find_parent,-find_all),-recursive-동작-비교" data-toc-modified-id="부모자식-tag-선택하기(find_parent,-find_all),-recursive-동작-비교-1.6.3">부모자식 tag 선택하기(find_parent, find_all), recursive 동작 비교</a></span></li><li><span><a href="#좌우-tag-선택하기-(find_next_sibling,-find_previous_-sibling)" data-toc-modified-id="좌우-tag-선택하기-(find_next_sibling,-find_previous_-sibling)-1.6.4">좌우 tag 선택하기 (find_next_sibling, find_previous_ sibling)</a></span></li><li><span><a href="#선물-가격정보-추출하기" data-toc-modified-id="선물-가격정보-추출하기-1.6.5">선물 가격정보 추출하기</a></span></li></ul></li><li><span><a href="#구글-검색결과-추출하기" data-toc-modified-id="구글-검색결과-추출하기-1.7">구글 검색결과 추출하기</a></span></li><li><span><a href="#학습-차원에서-DOM-Tree를-위에서-부터-차례로-탐색해봅니다.-(숙제)" data-toc-modified-id="학습-차원에서-DOM-Tree를-위에서-부터-차례로-탐색해봅니다.-(숙제)-1.8">학습 차원에서 DOM Tree를 위에서 부터 차례로 탐색해봅니다. (숙제)</a></span></li><li><span><a href="#네이버,-다음-검색결과-추출하기-(숙제)" data-toc-modified-id="네이버,-다음-검색결과-추출하기-(숙제)-1.9">네이버, 다음 검색결과 추출하기 (숙제)</a></span></li></ul></li></ul></div>

"05.웹 크롤링 - DOM.pdf"(day5) 강의 내용에 대한 실습 코드 입니다.

## BeautifulSoup

이전 강의에서 정의한 getDownload 함수를 사용해서 웹페이지를 다운로드 해봅니다.

In [2]:
import requests

headers = {"user-agent": "Mozilla/5.0"}
base_retries = 3

def getDownload(url, param=None, retries=base_retries):
    resp = None

    try:
        resp = requests.get(url, params=param, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retries > 0:
            print("Retries: {0}".format(base_retries - retries + 1))
            return getDownload(url, param, retries - 1)
        else:
            print(resp.status_code)
            print(resp.reason)
            print(resp.headers)
    
    return resp

In [3]:
url = "https://www.google.com/search"
param = {"q":"파이썬"}

html = getDownload(url, param=param)
html.text

'<!doctype html><html itemscope="" itemtype="http://schema.org/SearchResultsPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><noscript><meta content="0;url=/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&amp;gbv=1&amp;sei=OiqCXLbAMpXXhwO1sLnQDA" http-equiv="refresh"><style>table,div,span,p{display:none}</style><div style="display:block">몇 초 안에 이동하지 않는 경우 <a href="/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&amp;gbv=1&amp;sei=OiqCXLbAMpXXhwO1sLnQDA">여기</a>를 클릭하세요.</div></noscript><title>파이썬 - Google 검색</title><style>#gb{font:13px/27px Arial,sans-serif;height:30px}#gbz,#gbg{position:absolute;white-space:nowrap;top:0;height:30px;z-index:1000}#gbz{left:0;padding-left:4px}#gbg{right:0;padding-right:5px}#gbs{background:transparent;position:absolute;top:-999px;visibility:hidden;z-index:998;right:0}.gbto #gbs{background:#fff}#gbx3,#gbx4{background-color:#2d2d2d;background-image:

In [4]:
type(html)    # Response 객체

requests.models.Response

In [5]:
type(html.text)  # string

str

다운로드 한 html.text가 위에서 보는 것처럼 string이기 때문에 직접 조작하기 어렵습니다.

DOM tree를 구성해서 조작하기 쉽도록 해주는 **`BeautifulSoup`** 패키지를 사용 합니다.

### BeautifulSoup 설치

In [6]:
# !conda install beautifulsoup4
# !pip install beautifulsoup4

### BeautifulSoup import 하기

설치한 패키지명은 `beautifulsoup4` 이지만 import 할 패키지명은 `bs4` 입니다.

In [7]:
from bs4 import BeautifulSoup

### BeautifulSoup 객체 생성 및 parser 사용

HTML은 XML과 달리 **`Not Well-formed`** 이기 때문에 자동으로 분석하기 어렵다.

HTML 웹 페이지의 흔한 오류들...
- 닫는 태그가 없는 경우가 있다.
- elements가 overlap 된 경우가 있다.
- 여는 태그와 닫는 태그의 case(대소문자)가 일치하지 않는 경우가 있다.

속도와 오류처리 성능 등을 고려한 `parser` 선택이 중요하다. ( **`lxml parser`** 를 추천함 )
- python의 html.parser
- lxml의 html.parser
- lxml의 xml.parser
- html5lib

parser 들을 사용 해보자.

In [8]:
# 테스트를 위한 간단한 html 페이지를 만든다. (Well-formed)
html = """
<document>
<html>
    <head></head>
    <body>
        <div>
            <p>
                <a>Go to page</a>
            </p>
        </div>
    </body>
</document>
"""

In [9]:
dom = BeautifulSoup(html, "lxml")    # features 파라미터에 parser를 지정 (lxml parser 사용)

In [10]:
type(dom)

bs4.BeautifulSoup

In [11]:
dom.html.head   # 태그로 접근 가능함

In [12]:
type(dom.html)

bs4.element.Tag

In [13]:
type(dom.html.body)

bs4.element.Tag

BeautifulSoup은 DOM Tree를 반환하며, 이 객체는 tag 명칭과 값의 쌍으로 이루어진 **`Tag 객체`** 들로 이루어져 있다.

In [14]:
dom.html

<html><body><document>
<div>
<p>
<a>Go to page</a>
</p>
</div>
</document>
</body></html>

In [15]:
dom.prettify()   # 들여쓰기를 해서 보기좋게 출력함

'<html>\n <body>\n  <document>\n   <div>\n    <p>\n     <a>\n      Go to page\n     </a>\n    </p>\n   </div>\n  </document>\n </body>\n</html>'

In [16]:
dom.html.body.div.p.a

<a>Go to page</a>

In [17]:
dom.html.body.div.p.a.text

'Go to page'

In [18]:
dom.a.text

'Go to page'

추가적으로 `name`, `attr`, `text` 등 속성도 사용해볼것 (dom.a.attr, ...)

BeautifulSoup의 DOM Tree는 위에서 처럼, <br>
Tag를 chain 형태로 연결해서 단계적으로 접근하거나, 특정 Tag를 바로 접근도 가능하다.

### jupyter notebook Magic Command : whos

In [19]:
%whos

Variable        Type             Data/Info
------------------------------------------
BeautifulSoup   type             <class 'bs4.BeautifulSoup'>
autopep8        module           <module 'autopep8' from '<...>te-packages/autopep8.py'>
base_retries    int              3
dom             BeautifulSoup    <html><body><document>\n<<...>document>\n</body></html>
getDownload     function         <function getDownload at 0x10395cb70>
headers         dict             n=1
html            str              \n<document>\n<html>\n   <...>   </body>\n</document>\n
json            module           <module 'json' from '/Use<...>hon3.6/json/__init__.py'>
param           dict             n=1
requests        module           <module 'requests' from '<...>es/requests/__init__.py'>
url             str              https://www.google.com/search


### Selector 기본

In [20]:
# <a> 태그를 하나 더 추가해본다.
html = """
<document>
<html>
    <head></head>
    <body>
        <div>
            <p>
                <a>Go to page1</a>
                <a>Go to page2</a>
            </p>
        </div>
    </body>
</document>
"""

In [21]:
dom = BeautifulSoup(html, "lxml")

In [22]:
dom.a

<a>Go to page1</a>

In [23]:
dom.find('a')

<a>Go to page1</a>

In [24]:
dom.find_all('a')   # BeautifulSoup이 버전업 되면서, findAll() 형태의 메서드는 없어지고, find_all() 형태의 메서드로 대체되었음

[<a>Go to page1</a>, <a>Go to page2</a>]

위의 방법으로는 두번째 `<a>` 태그 만 가져오지는 못한다. 

태그에 `id`와 `class`를 추가해본다.

In [25]:
html = """
<document>
<html>
    <head></head>
    <body>
        <div id="result">
            <p class="row">
                <a class="red">Go to page1</a>
                <a class="blue">Go to page2</a>
            </p>
        </div>
    </body>
</document>
"""

In [26]:
dom = BeautifulSoup(html, "lxml")

In [27]:
import bs4   # bs4에서는 기본적으로 Tag의 속성들을 제어할 수 있는 다양한 메서드를 제공함

# bs4.element.Tag.get()
# bs4.element.Tag == DOM.Elements

In [28]:
dom.a, dom.find('a')

(<a class="red">Go to page1</a>, <a class="red">Go to page1</a>)

In [29]:
for tag in dom.find_all('a'):
    print(tag.name, tag["class"], tag.text)

a ['red'] Go to page1
a ['blue'] Go to page2


태그에 ID와 Class를 지정해줬기 때문에 아래처럼 특정 element에 직접 접근이 가능하다.

In [30]:
dom.find_all('a', {"class":"blue"})

[<a class="blue">Go to page2</a>]

In [31]:
dom.find_all(["div", "a"])   # tag는 여러개를 한꺼번에 지정할 수 있다.(튜플, 리스트, Set 등)

[<div id="result">
 <p class="row">
 <a class="red">Go to page1</a>
 <a class="blue">Go to page2</a>
 </p>
 </div>, <a class="red">Go to page1</a>, <a class="blue">Go to page2</a>]

In [32]:
dom.find_all("", {"id":"result"})     # tag를 지정하지 않고, 속성만으로 찾을 수도 있다.

[<div id="result">
 <p class="row">
 <a class="red">Go to page1</a>
 <a class="blue">Go to page2</a>
 </p>
 </div>]

In [33]:
dom.find_all("a", recursive=False)
# dom.find_all("a", recursive=True)    # default는 True

[]

find_all() 함수에 recursive=False로 지정하면 특정 태그를 direct로 찾지 못한다. direct children 만 찾을 수 있다.

In [34]:
dom.find_all("html", recursive=False)

[<html><body><document>
 <div id="result">
 <p class="row">
 <a class="red">Go to page1</a>
 <a class="blue">Go to page2</a>
 </p>
 </div>
 </document>
 </body></html>]

### Selector 실습

http://pythonscraping.com/pages/page3.html 사이트에서 Selector 실습하기

![DOM Exercises](./images/selector_exercises.png)

In [35]:
url = "http://pythonscraping.com/pages/page3.html"
html = getDownload(url, param=param)

#### html.parser

In [36]:
dom = BeautifulSoup(html.content, "html.parser")

In [37]:
footer = dom.find("div", {"id":"footer"})
footer

<div id="footer">
© Totally Normal Gifts, Inc. <br/>
+234 (617) 863-0736
</div>

In [38]:
parent = footer.find_parent()
parent.name, parent.attrs

('[document]', {})

#### lxml parser

In [39]:
dom = BeautifulSoup(html.content, "lxml")

In [40]:
footer = dom.find("div", {"id":"footer"})
footer

<div id="footer">
© Totally Normal Gifts, Inc. <br/>
+234 (617) 863-0736
</div>

In [41]:
parent = footer.find_parent()
parent.name, parent.attrs

('div', {'id': 'wrapper'})

In [42]:
parents = footer.find_parents()
for row in parents:
    print(row.name, row.attrs)

div {'id': 'wrapper'}
body {}
html {}
[document] {}


lxml parser는 find_parent() 결과가 정상적이지만, html.parser는 tag를 찾지 못했습니다.<br>
parser 마다 특성이 있습니다. 여기서는 lxml parser를 사용합니다.

#### 부모자식 tag 선택하기(find_parent, find_all), recursive 동작 비교

In [43]:
children = parent.find_all()    # default는 recursive=False
for row in children:
    print(row.name, row.attrs)

img {'src': '../img/gifts/logo.jpg', 'style': 'float:left;'}
h1 {}
div {'id': 'content'}
p {}
br {}
br {}
table {'id': 'giftList'}
tr {}
th {}
th {}
th {}
th {}
tr {'id': 'gift1', 'class': ['gift']}
td {}
td {}
span {'class': ['excitingNote']}
td {}
td {}
img {'src': '../img/gifts/img1.jpg'}
tr {'id': 'gift2', 'class': ['gift']}
td {}
td {}
span {'class': ['excitingNote']}
td {}
td {}
img {'src': '../img/gifts/img2.jpg'}
tr {'id': 'gift3', 'class': ['gift']}
td {}
td {}
span {'class': ['excitingNote']}
td {}
td {}
img {'src': '../img/gifts/img3.jpg'}
tr {'id': 'gift4', 'class': ['gift']}
td {}
td {}
span {'class': ['excitingNote']}
td {}
td {}
img {'src': '../img/gifts/img4.jpg'}
tr {'id': 'gift5', 'class': ['gift']}
td {}
td {}
span {'class': ['excitingNote']}
td {}
td {}
img {'src': '../img/gifts/img6.jpg'}
div {'id': 'footer'}
br {}


위에서 처럼  자식 노드에서 부모 노드로 이동 후 전체 태그를 찾는 `find_all()` 메서드 결과를 보면, <br>
원래의 children tag의 하부 tree 뿐 아니라 형제 Tree의 하부 까지 모두 찾는다.

`recursive=False`로 지정하면, 원래의 children tag의 하부 Tree 만 찾는다.

In [44]:
children = parent.find_all(recursive=False)
for row in children:
    print(row.name, row.attrs)

img {'src': '../img/gifts/logo.jpg', 'style': 'float:left;'}
h1 {}
div {'id': 'content'}
table {'id': 'giftList'}
div {'id': 'footer'}


#### 좌우 tag 선택하기 (find_next_sibling, find_previous_ sibling)

In [45]:
descendants = parent.find_all()

for row in descendants:
    print(row.name, row.attrs)

img {'src': '../img/gifts/logo.jpg', 'style': 'float:left;'}
h1 {}
div {'id': 'content'}
p {}
br {}
br {}
table {'id': 'giftList'}
tr {}
th {}
th {}
th {}
th {}
tr {'id': 'gift1', 'class': ['gift']}
td {}
td {}
span {'class': ['excitingNote']}
td {}
td {}
img {'src': '../img/gifts/img1.jpg'}
tr {'id': 'gift2', 'class': ['gift']}
td {}
td {}
span {'class': ['excitingNote']}
td {}
td {}
img {'src': '../img/gifts/img2.jpg'}
tr {'id': 'gift3', 'class': ['gift']}
td {}
td {}
span {'class': ['excitingNote']}
td {}
td {}
img {'src': '../img/gifts/img3.jpg'}
tr {'id': 'gift4', 'class': ['gift']}
td {}
td {}
span {'class': ['excitingNote']}
td {}
td {}
img {'src': '../img/gifts/img4.jpg'}
tr {'id': 'gift5', 'class': ['gift']}
td {}
td {}
span {'class': ['excitingNote']}
td {}
td {}
img {'src': '../img/gifts/img6.jpg'}
div {'id': 'footer'}
br {}


In [46]:
divChildren = children[2].find_all(recursive=False)

for row in divChildren:
    print(row.name, row.attrs, row.text)

p {} 
We haven't figured out how to make online shopping carts yet, but you can send us a check to:
123 Main St.
Abuja, Nigeria
We will then send your totally amazing gift, pronto! Please include an extra $5.00 for gift wrapping.


In [47]:
divTag = children[2]

# children[1] == divTag.find_previous_sibling()
children[1].name, divTag.find_previous_sibling().name

('h1', 'h1')

In [48]:
children[3] == divTag.find_next_sibling()

True

#### 선물 가격정보 추출하기

찾고자 하는 정보를 바로 찾을 수 있으면 좋지만, <br>
그렇지 않으면 상위의 태그 중에 id나 class가 명확히 구분되는 태그를 찾아서 tree를 탐색한다.

선물 리스트가 테이블에 담겨있고, 각각의 선물 리스트는 "gift"라는 class명을 가지고 있다.<br>
각각의 선물 리스트는 4개의 td 태그로 되어 있고, 3번째에 cost 정보가 포함되어 있다.

In [49]:
gifts = dom.find_all("", {"class":"gift"})

costs = []

for gift in gifts:
    tds = gift.find_all(recursive=False)    # 각각의 gift별 td 태그 리스트
    costs.append(tds[2].text.strip())

print(costs)

['$15.00', '$10,000.52', '$10,005.00', '$0.50', '$1.50']


### 구글 검색결과 추출하기

구글 검색을 통해 "파이썬 자습서" 라는 문자열의 검색결과 중 제목 부분을 추출해봅니다.

`우선 브라우저의 "페이지 소스 보기" 또는 "검사" 기능을 이용해서, 검색결과에서 찾고자 하는 문자열을 탐색 해봅니다.`

제목 자체의 class명은 검색 리스트 마다 다르지만,<br>
`제목 위쪽에 있는 div 태그의 class명이 "r"로 일정한 것을 발견할 수 있습니다.`  (찾기가 쉽지는 않습니다.)

In [50]:
url = "http://www.google.com/search"
param = {"q":"파이썬 자습서"}

html = getDownload(url, param)
dom = BeautifulSoup(html.text, "lxml")

for tag in dom.find_all("", {"class":"r"})[:3]:
    print(tag.prettify())

<h3 class="r">
 <a href="/url?q=https://docs.python.org/ko/3/tutorial/index.html&amp;sa=U&amp;ved=0ahUKEwjT3O_HkvLgAhXRfXAKHUxqBywQFggVMAA&amp;usg=AOvVaw28_54tQuRtBWrxC_Lf-sFt">
  <b>
   파이썬 자습서
  </b>
  —
  <b>
   Python
  </b>
  3.7.2 documentation
 </a>
</h3>

<h3 class="r">
 <a class="sla" href="/url?q=https://docs.python.org/ko/3/tutorial/appetite.html&amp;sa=U&amp;ved=0ahUKEwjT3O_HkvLgAhXRfXAKHUxqBywQjBAIGzAB&amp;usg=AOvVaw3lzzfjxeQHZp3YAo6i-T0t">
  1. 입맛 돋우기
 </a>
</h3>

<h3 class="r">
 <a class="sla" href="/url?q=https://docs.python.org/ko/3/tutorial/interpreter.html&amp;sa=U&amp;ved=0ahUKEwjT3O_HkvLgAhXRfXAKHUxqBywQjBAIHTAC&amp;usg=AOvVaw1-Mb0eHpDnxUTxwyDEF3bn">
  2. 파이썬 인터프리터 사용하기
 </a>
</h3>



`<h3 class="r"> 태그 하부에 <a> 태그의 text 부분에 제목이 보입니다.`

In [51]:
for tag in dom.find_all("", {"class":"r"}):
    print(tag.find().text)

파이썬 자습서 — Python 3.7.2 documentation
1. 입맛 돋우기
2. 파이썬 인터프리터 사용하기
프로그래머가 아닌 이들을 위한 파이썬 3 자습서 - 위키책
byte of python(한국어판 PDF) - SourceForge
파이썬 강좌 및 유용한 사이트 모음 : 네이버 블로그
Visual Studio 자습서 1단계의 Python 프로젝트 만들기 | Microsoft Docs
Python | 자습서, API, SDK, 설명서 | AWS 개발자 센터 - Amazon.com
무료 파이썬 자습서, eBook 및 전자책 - Blue Breeze
[리뷰] 파이썬 코딩 도장 - 브런치
파이썬 정복: 파이썬 개발에 필요한 기본 지식을 모두 담은 파이썬 입문서 - Google 도서 검색결과


브라우저의 검색된 결과와 비교해보면, 검색 링크가 있는 모든 제목이 출력되었습니다.

### 학습 차원에서 DOM Tree를 위에서 부터 차례로 탐색해봅니다. (숙제)

이번에는 브라우저 검색결과에 의존하지 않고, DOM Tree를 탐색하는 방법을 고민해보겠습니다.

tag 명칭과 클래스/ID를 추출해서 dom tree를 출력해주는 함수를 만들어 사용하면 좋을 듯 합니다.

전체 tree를 그려보고, 특정 문자열이 포함된 tag를 찾은 후 부모 tree를 재구성 해서 비교하면 ... 탐색을 자동화 하는 것도 가능하지 않은지???

In [55]:
# html의 전체 tag를 recursive 하게 돌면서, 전체 tree를 그려주는 함수 만들기


### 네이버, 다음 검색결과 추출하기 (숙제)